In [7]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import pyranges as pr
import itertools
import sys
sys.path.insert(0, '/Genomics/kocherlab/bjarnold/STARRseq/code/notebooks')

import functions as fn
from collections import defaultdict
import os
from pyfaidx import Fasta
from Bio.Seq import Seq



In [8]:
species = "Nmel"
bioreps = ['F1', 'F2', 'F3']

chr1_test = False
qval_threshold = 1 # 1.3 corresponds to pvalue of 0.05, 1 corresponds to pvalue of 0.1
macs_folddiff_threshold = 2
interval_size = 40



In [9]:
base_dir = "/Genomics/kocherlab/bjarnold/STARRseq/data"
wd = f"{base_dir}/peak_calling_snakemake_output/{species}"

# load MACS peak files
macs_raw_peaks_files = fn.get_files(f'{wd}/MACS2', "*_peaks.narrowPeak")
macs_dedup_peaks_files = fn.get_files(f'{wd}/MACS2_dedup', "*_peaks.narrowPeak")

print(macs_raw_peaks_files)
print(macs_dedup_peaks_files)

#load ref genome
ref_genome = f'{base_dir}/alternate_references/renamed/{species}.fasta'

['/Genomics/kocherlab/bjarnold/STARRseq/data/peak_calling_snakemake_output/Nmel/MACS2/Nmel-F1_peaks.narrowPeak', '/Genomics/kocherlab/bjarnold/STARRseq/data/peak_calling_snakemake_output/Nmel/MACS2/Nmel-F2_peaks.narrowPeak', '/Genomics/kocherlab/bjarnold/STARRseq/data/peak_calling_snakemake_output/Nmel/MACS2/Nmel-F3_peaks.narrowPeak']
['/Genomics/kocherlab/bjarnold/STARRseq/data/peak_calling_snakemake_output/Nmel/MACS2_dedup/Nmel-F1_peaks.narrowPeak', '/Genomics/kocherlab/bjarnold/STARRseq/data/peak_calling_snakemake_output/Nmel/MACS2_dedup/Nmel-F2_peaks.narrowPeak', '/Genomics/kocherlab/bjarnold/STARRseq/data/peak_calling_snakemake_output/Nmel/MACS2_dedup/Nmel-F3_peaks.narrowPeak']


In [10]:
macs_raw_dfs = fn.load_peak_caller_results(macs_raw_peaks_files, chr1_test) # list of macs dataframes
macs_dedup_dfs = fn.load_peak_caller_results(macs_dedup_peaks_files, chr1_test) # list of macs dataframes

# FILTER PEAK FILES BY SIGNIFICANCE, EFFECT SIZE
print('MACS')
macs_raw_dfs = fn.filter_by_sig_effect_size(macs_raw_dfs, qval_threshold, macs_folddiff_threshold)
macs_dedup_dfs = fn.filter_by_sig_effect_size(macs_dedup_dfs, qval_threshold, macs_folddiff_threshold)

print([len(df) for df in macs_raw_dfs])
print([len(df) for df in macs_dedup_dfs])



MACS
before filtering: 103313
after filtering: 34129
before filtering: 101930
after filtering: 31722
before filtering: 104845
after filtering: 36205
before filtering: 9645
after filtering: 4306
before filtering: 13153
after filtering: 5499
before filtering: 12217
after filtering: 4899
[34129, 31722, 36205]
[4306, 5499, 4899]


In [11]:
# can't read with pandas because variable number of columns; some rows marked with an asterisk if the repeat overlaps with another higher-scoring repeat
# load by reading in all lines instead, then importing into table
# then use pyranges to merge overlapping intervals
# repeat_pr = fn.load_repeat_modeler_intervals(repeat_file)

# open reference genome ref_genome, count the length of each chromosome, and store in a dictionary
fasta_sequences = Fasta(ref_genome)

scaff_lengths = defaultdict(int)
# Iterate over the sequences and print their names and lengths
for sequence_name in fasta_sequences.keys():
    scaff_lengths[sequence_name] = len(fasta_sequences[sequence_name])


In [12]:
def get_summit_intervals(macs_dfs):
    
    for m in macs_dfs:
        # convert peak/summit coordinate into an interval to overlap with pyranges
        m['peak_coord_end'] = m['peak_coord'] + int(interval_size/2)
        m['peak_coord_start'] = m['peak_coord'] - int(interval_size/2)

    # gather relevant columns, put in new list of dataframes
    macs_summit_dfs = [m[['Chromosome', 'peak_coord_start', 'peak_coord_end', 'signalValue']] for m in macs_dfs]

    # rename columns to match pyranges format
    for m in macs_summit_dfs:
        m.columns = ['Chromosome', 'Start', 'End', 'signalValue']
        m.reset_index(inplace=True, drop=True) # THIS IS REALLY IMPORTANT, OTHERWISE FETCHING SEQUENCES RETURNS NANS

    # convert df to pyranges object
    # macs_summit_prs = [pr.PyRanges(m) for m in macs_summit_dfs]

    return macs_summit_dfs

# extend intervals around summit
macs_raw_summit_intervals_df = get_summit_intervals(macs_raw_dfs)
macs_dedup_summit_intervals_df = get_summit_intervals(macs_dedup_dfs)

# grab sequences from reference genome within intervals
for m in macs_raw_summit_intervals_df:
    m.loc[:,'Sequence'] = pr.get_sequence(pr.PyRanges(m), ref_genome)
for m in macs_dedup_summit_intervals_df:
    m.loc[:,'Sequence'] = pr.get_sequence(pr.PyRanges(m), ref_genome)


/tmp/ipykernel_2257796/2266070714.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m.loc[:,'Sequence'] = pr.get_sequence(pr.PyRanges(m), ref_genome)
/tmp/ipykernel_2257796/2266070714.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  m.loc[:,'Sequence'] = pr.get_sequence(pr.PyRanges(m), ref_genome)
/tmp/ipykernel_2257796/2266070714.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [24]:
tmp = macs_raw_summit_intervals_df[0]
# print(tmp.Chromosome.unique())
tmp[tmp['Chromosome'] == 'NMEL_chr_2'].head()

,Chromosome,Start,End,signalValue,Sequence
1605,NMEL_chr_1,14192628,14192668,3.46729,TCGTCAGATACATAAAATCTTAGCAGTGAGGTTGACGAAA
1606,NMEL_chr_1,14197608,14197648,2.38467,AATCAATAACTCAAAAACTACTCAATCAATCTTGCTGAAA
1607,NMEL_chr_1,14202616,14202656,2.03423,TCTACTTTCAGCGATACCAATGAAACCATATTGTTCGAAG
1608,NMEL_chr_1,14205308,14205348,3.86016,GCGAGCAATAATTTATTTGATTATTGAAAGAAACTTTCTG
1609,NMEL_chr_1,14212178,14212218,5.24032,GAGCCCTGAATATTTTTTACCATAGCAGAAATAAAAGCCC


In [14]:
tmp = tmp[['Chromosome', 'Start', 'End']]
# write tmp to a tab separated file
tmp.to_csv(f'./tmp.txt', sep='\t', index=False)

In [15]:
for m in macs_dedup_summit_intervals_df:
    print(m.isna().sum())

Chromosome     0
Start          0
End            0
signalValue    0
Sequence       0
dtype: int64
Chromosome     0
Start          0
End            0
signalValue    0
Sequence       0
dtype: int64
Chromosome     0
Start          0
End            0
signalValue    0
Sequence       0
dtype: int64


In [16]:
for m in macs_raw_summit_intervals_df:
    print(m.isna().sum())

Chromosome     0
Start          0
End            0
signalValue    0
Sequence       0
dtype: int64
Chromosome     0
Start          0
End            0
signalValue    0
Sequence       0
dtype: int64
Chromosome     0
Start          0
End            0
signalValue    0
Sequence       0
dtype: int64


In [17]:
macs_dedup_summit_intervals_df[2].tail(50)

,Chromosome,Start,End,signalValue,Sequence
4849,NMEL_unplaced_33,9630,9670,2.32957,TTTCATGCTCGCAACACTCGCAAACGATCTTCCATTCATA
4850,NMEL_unplaced_35,8963,9003,2.69679,TTCTTACATTCATCCAATCGTCCCTTAAGTCTCTCATTCT
4851,NMEL_unplaced_38,5687,5727,2.69679,ACGCATCGACCGAATGATTCATTTCGATGATTACTGACCG
4852,NMEL_unplaced_393,673,713,3.23615,ATTATGATCCTCTCTCGGCAGTGATGTCGCACTGTCCAGA
4853,NMEL_unplaced_400,490,530,2.69679,ATGGGTGAATATGAGCAGTGTGTTGTGAAATTGATGTGTG
4854,NMEL_unplaced_42,11257,11297,2.51701,TGAATCTGTGCAGCGGGAACGACCTCGGACTACCAGGGGC
4855,NMEL_unplaced_439,1720,1760,2.87658,ACCTTCTCTCCTTGGGCAAATGAGATCGGCCGTGCACGGG
4856,NMEL_unplaced_443,838,878,2.51701,CCGAGAAGCGACCTGGATGTGTGTTATGCGAGTTCACTTT
4857,NMEL_unplaced_443,1977,2017,2.51701,GCGATCTGGATGTGTGTTATGCGAGTACACATTGATCTTA
4858,NMEL_unplaced_447,2452,2492,3.77551,TCGTAGCGGTCTTTGAACGCGAGGCCCCGCCCCGGATAGA


In [18]:
scaff_lengths['NMEL_unplaced_52']

8404

In [19]:
macs_dedup_summit_intervals_df[2].isna().sum()

Chromosome     0
Start          0
End            0
signalValue    0
Sequence       0
dtype: int64

In [20]:
tmp = macs_dedup_summit_intervals_df[2]

In [21]:
# kmer_df['sequence'] = pr.get_sequence(pr.PyRanges(kmer_df), ref_genome)
